# Creating command-line Tools

In [1]:
from ctapipe.core import Tool, Component, TelescopeComponent
from ctapipe.core.traits import (
    Integer, Float, List, Dict, Unicode,
    TraitError, observe, FloatTelescopeParameter,
    Path,
)
import logging
from time import sleep
from astropy import units as u
from ctapipe.utils import get_dataset_path

In [2]:
GAMMA_FILE = get_dataset_path('gamma_test.simtel.gz')

see https://github.com/ipython/traitlets/blob/master/examples/myapp.py

## Setup:

Create a few `Component`s that we will use later in a `Tool`:

In [3]:
class MyComponent(Component):
    """ A Component that does stuff """

    value = Integer(default_value=-1, help="Value to use").tag(config=True)

    def do_thing(self):
        self.log.debug("Did thing")


# in order to have 2 of the same components at once
class SecondaryMyComponent(MyComponent):
    pass


class AdvancedComponent(Component):
    """ An advanced technique """

    value1 = Integer(default_value=-1, help="Value to use").tag(config=True)
    infile = Path(
        help="input file name", 
        exists=None,  # set to True to require existing, False for requiring non-existing
        directory_ok=False,
    ).tag(config=True)
    outfile = Path(
        help="output file name",
        exists=False, directory_ok=False
    ).tag(config=True)
    
    def __init__(self, config=None, parent=None, **kwargs):
        super().__init__(config=config, parent=parent, **kwargs)
        # components can have sub components, but these must have 
        # then parent=self as argument and be assigned as member
        # so the full config can be received later
        self.subcompent = MyComponent(parent=self)

    @observe("outfile")
    def on_outfile_changed(self, change):
        self.log.warning("Outfile was changed to '{}'".format(change))


class TelescopeWiseComponent(TelescopeComponent):
    """ a component that contains parameters that are per-telescope configurable """

    param = FloatTelescopeParameter(
        help="Something configurable with telescope patterns", default_value=5.0
    ).tag(config=True)

In [4]:
MyComponent()

value,-1,Value to use (default: -1)


In [5]:
AdvancedComponent(infile="test.foo", outfile="out.foo")

Outfile was changed to '{'name': 'outfile', 'old': traitlets.Undefined, 'new': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/out.foo'), 'owner': <__main__.AdvancedComponent object at 0x7f5d96899520>, 'type': 'change'}'


infile,/home/runner/work/ctapipe/ctapipe/docs/examples/test.foo,input file name (default: traitlets.Undefined)
outfile,/home/runner/work/ctapipe/ctapipe/docs/examples/out.foo,output file name (default: traitlets.Undefined)
value1,-1,Value to use (default: -1)


`TelescopeComponents` need to have a subarray given to them in order to work (since they need one to turn a `TelescopeParameter` into a concrete list of values for each telescope.  Here we will give a dummy one:

In [6]:
from ctapipe.instrument import SubarrayDescription, TelescopeDescription

subarray = SubarrayDescription(
    "Junk",
    tel_positions={1: (0.0, 0.0, 0.0) * u.m, 2: (1.0, 1.0, 0.0) * u.m},
    tel_descriptions={
        1: TelescopeDescription.from_name("LST", "LSTCam"),
        2: TelescopeDescription.from_name("MST", "NectarCam"),
    },
)
subarray.info()

Subarray : Junk
Num Tels : 2
Footprint: 0.00 km2

       Type       Count Tel IDs
----------------- ----- -------
MST_MST_NectarCam     1 2      
   LST_LST_LSTCam     1 1      


In [7]:
TelescopeWiseComponent(subarray=subarray)

param,"[('type', '*', 5.0)]",Something configurable with telescope patterns (default: traitlets.Undefined)


This TelescopeParameters can then be set using a list of patterns like:
```python
component.param = [ 
    ("type", "LST*",3.0), 
    ("type", "MST*", 2.0), 
    (id, 25, 4.0) 
]
```

These get translated into per-telescope-id values once the subarray is registered.  After that one acccess the per-telescope id values via:
```python
component.param.tel[tel_id]
```

## Now create an executable Tool that contains the Components

In [8]:
class MyTool(Tool):
    name="mytool"
    description="do some things and stuff"
    aliases = dict(
        infile='AdvancedComponent.infile',
        outfile='AdvancedComponent.outfile',
        iterations='MyTool.iterations'
    )

    # Which classes are registered for configuration
    classes = [MyComponent, AdvancedComponent, SecondaryMyComponent, TelescopeWiseComponent]

    # local configuration parameters
    iterations = Integer(5,help="Number of times to run",allow_none=False).tag(config=True)

    def setup(self):
        self.comp = MyComponent(parent=self)
        self.comp2 = SecondaryMyComponent(parent=self)
        self.comp3 = TelescopeWiseComponent(parent=self, subarray=subarray)
        self.advanced = AdvancedComponent(parent=self)


    def start(self):
        self.log.info("Performing {} iterations...".format(self.iterations))
        for ii in range(self.iterations):
            self.log.info("ITERATION {}".format(ii))
            self.comp.do_thing()
            self.comp2.do_thing()
            sleep(0.1)
            
    def finish(self):
        self.log.warning("Shutting down.")
    

## Get Help info

The following allows you to print the help info within a Jupyter notebook, but this same inforamtion would be displayed if the user types:
```
  mytool --help
```

In [9]:
tool = MyTool()
tool

config_files,[],(default: traitlets.Undefined)
iterations,5,Number of times to run (default: 5)
log_config,{},(default: traitlets.Undefined)
log_datefmt,%Y-%m-%d %H:%M:%S,The date format used by logging formatters for %(asctime)s (default: %Y-%m-%d %H:%M:%S)
log_file,None,Filename for the log (default: None)
log_file_level,INFO,Logging Level for File Logging (default: INFO)
log_format,[%(name)s]%(highlevel)s %(message)s,The Logging format template (default: [%(name)s]%(highlevel)s %(message)s)
log_level,30,Set the log level by value or name. (default: 30)
logging_config,{},"Configure additional log handlers. The default stderr logs handler is configured by the log_level, log_datefmt and log_format settings. This configuration can be used to configure additional handlers (e.g. to output the log to a file) or for finer control over the default handlers. If provided this should be a logging configuration dictionary, for more information see: https://docs.python.org/3/library/logging.config.html#logging-config-dictschema This dictionary is merged with the base logging configuration which defines the following: * A logging formatter intended for interactive use called ``console``. * A logging handler that writes to stderr called ``console`` which uses the formatter ``console``. * A logger with the name of this application set to ``DEBUG`` level. This example adds a new handler that writes to a file: .. code-block:: python c.Application.logging_config = { 'handlers': { 'file': { 'class': 'logging.FileHandler', 'level': 'DEBUG', 'filename': '', } }, 'loggers': { '': { 'level': 'DEBUG', # NOTE: if you don't list the default ""console"" # handler here then it will be disabled 'handlers': ['console', 'file'], }, } } (default: traitlets.Undefined)"
provenance_log,/home/runner/work/ctapipe/ctapipe/docs/examples/mytool.provenance.log,(default: traitlets.Undefined)
quiet,False,(default: False)


In [10]:
tool.print_help()

do some things and stuff

Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    Set log-level to debug, for the most verbose logging.
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
-q, --quiet
    Disable console logging.
    Equivalent to: [--Tool.quiet=True]
-v, --verbose
    Set log level to DEBUG
    Equivalent to: [--Tool.log_level=DEBUG]
-c, --config=<list-item-1>...
    Default: []
    Equivalent to: [--Tool.config_files]
--log-level=<Enum>
    Set the log level by value or name.
    Choices: any of [0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO'

The following  is equivalant to the user typing `mytool --help-all`

In [11]:
tool.print_help(classes=True)

do some things and stuff

Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    Set log-level to debug, for the most verbose logging.
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
-q, --quiet
    Disable console logging.
    Equivalent to: [--Tool.quiet=True]
-v, --verbose
    Set log level to DEBUG
    Equivalent to: [--Tool.log_level=DEBUG]
-c, --config=<list-item-1>...
    Default: []
    Equivalent to: [--Tool.config_files]
--log-level=<Enum>
    Set the log level by value or name.
    Choices: any of [0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO'

## Run the tool

here we pass in argv since it is a Notebook, but if argv is not specified it's read from `sys.argv`, so the following is the same as running:

```sh
mytool --log_level=INFO --infile gamma_test.simtel.gz --iterations=3
```

As Tools are intended to be exectutables, they are raising `SystemExit` on exit.
Here, we use them to demonstrate how it would work, so we catch the `SystemExit`.

In [12]:
try:
    tool.run(argv=['--infile',  str(GAMMA_FILE), '--outfile', 'out.csv'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2022-07-04 14:49:23,742 WARNING [__main__.mytool.AdvancedComponent] (539495868.on_outfile_changed): Outfile was changed to '{'name': 'outfile', 'old': traitlets.Undefined, 'new': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/out.csv'), 'owner': <__main__.AdvancedComponent object at 0x7f5d945cb580>, 'type': 'change'}'


2022-07-04 14:49:24,247 WARNING [__main__.mytool] (1473845174.finish): Shutting down.


In [13]:
tool.log_format = "%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s" 


try:
    tool.run(argv=['--log-level','INFO','--infile', str(GAMMA_FILE), '--outfile', 'out.csv', '--iterations','3'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2022-07-04 14:49:24,277 INFO [__main__.mytool] (tool.initialize): Loading config from '[]'


2022-07-04 14:49:24,282 INFO [__main__.mytool] (tool.initialize): ctapipe version 0.15.1.dev142+ge09acbd4


2022-07-04 14:49:24,283 INFO [__main__.mytool] (tool.run): Starting: mytool


2022-07-04 14:49:24,297 WARNING [__main__.mytool.AdvancedComponent] (539495868.on_outfile_changed): Outfile was changed to '{'name': 'outfile', 'old': traitlets.Undefined, 'new': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/out.csv'), 'owner': <__main__.AdvancedComponent object at 0x7f5d9453f070>, 'type': 'change'}'


2022-07-04 14:49:24,300 INFO [__main__.mytool] (1473845174.start): Performing 3 iterations...


...

2022-07-04 14:49:24,506 INFO [__main__.mytool] (1473845174.start): ITERATION 2


2022-07-04 14:49:24,608 WARNING [__main__.mytool] (1473845174.finish): Shutting down.


2022-07-04 14:49:24,611 INFO [__main__.mytool] (tool.run): Finished: mytool


2022-07-04 14:49:24,616 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:24,618 INFO [__main__.mytool] (tool.write_provenance): Output: 


here we change the log-level to DEBUG:

In [14]:
try:
    tool.run(argv=['--log-level','DEBUG','--infile', str(GAMMA_FILE), '--outfile', 'out.csv'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2022-07-04 14:49:24,640 INFO [__main__.mytool] (tool.initialize): Loading config from '[]'


2022-07-04 14:49:24,643 DEBUG [__main__.mytool] (application._config_changed): Config changed: {'AdvancedComponent': {'infile': '/home/runner/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4/gamma_test.simtel.gz', 'outfile': 'out.csv'}, 'attach_subarray': <LazyConfigValue {}>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


2022-07-04 14:49:24,646 INFO [__main__.mytool] (tool.initialize): ctapipe version 0.15.1.dev142+ge09acbd4


2022-07-04 14:49:24,648 INFO [__main__.mytool] (tool.run): Starting: mytool


2022-07-04 14:49:24,661 WARNING [__main__.mytool.AdvancedComponent] (539495868.on_outfile_changed): Outfile was changed to '{'name': 'outfile', 'old': traitlets.Undefined, 'new': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/out.csv'), 'owner': <__main__.AdvancedComponent object at 0x7f5d94511550>, 'type': 'change'}'


...

2022-07-04 14:49:24,985 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:24,988 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:24,989 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:24,992 DEBUG [__main__.mytool] (tool.write_provenance): PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "d909fb92-6c63-4afd-a6af-21b59852e3ed",
      "start": {
         "time_utc": "2022-07-04T14:49:23.726"
      },
      "stop": {
         "time_utc": "2022-07-04T14:49:24.250"
      },
      "system": {
         "ctapipe_version": "0.15.1.dev142+ge09acbd4",
         "ctapipe_resources_version": "not installed",
         "eventio_version": "1.9.1",
         "ctapipe_svc_path": null,
         "executable": "/opt/hostedtoolcache/Python/3.8.13/x64/bin/python",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "ELF",
            "machine": "x86_64",
            "processor": "x86_64",
            "node": "fv-az173-750",
            "version": "#37~20.04.1-Ubuntu SMP Mon Jun 13 22:51:01 UTC 2022",
            "system": "Linux",
            "release": "5.13.0-1031-azure",
            "libcver": [


2022-07-04 14:49:25,000 DEBUG [__main__.mytool] (application.exit): Exiting application: mytool


you can also set parameters directly in the class, rather than using the argument/configfile parser. This is useful if you are calling the Tool from a script rather than the command-line

In [15]:
tool.iterations = 1
tool.log_level = 0

try:
    tool.run(['--infile', str(GAMMA_FILE), '--outfile', 'out.csv'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2022-07-04 14:49:25,019 DEBUG [__main__.mytool] (application._config_changed): Config changed: {'AdvancedComponent': {'infile': '/home/runner/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4/gamma_test.simtel.gz', 'outfile': 'out.csv'}, 'attach_subarray': <LazyConfigValue {}>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


2022-07-04 14:49:25,022 INFO [__main__.mytool] (tool.initialize): Loading config from '[]'


2022-07-04 14:49:25,025 DEBUG [__main__.mytool] (application._config_changed): Config changed: {'AdvancedComponent': {'infile': '/home/runner/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4/gamma_test.simtel.gz', 'outfile': 'out.csv'}, 'attach_subarray': <LazyConfigValue {}>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


2022-07-04 14:49:25,027 INFO [__main__.mytool] (tool.initialize): ctapipe version 0.15.1.dev142+ge09acbd4


2022-07-04 14:49:25,029 INFO [__main__.mytool] (tool.run): Starting: mytool


...

2022-07-04 14:49:25,369 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:25,370 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:25,370 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:25,374 DEBUG [__main__.mytool] (tool.write_provenance): PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "d909fb92-6c63-4afd-a6af-21b59852e3ed",
      "start": {
         "time_utc": "2022-07-04T14:49:23.726"
      },
      "stop": {
         "time_utc": "2022-07-04T14:49:24.250"
      },
      "system": {
         "ctapipe_version": "0.15.1.dev142+ge09acbd4",
         "ctapipe_resources_version": "not installed",
         "eventio_version": "1.9.1",
         "ctapipe_svc_path": null,
         "executable": "/opt/hostedtoolcache/Python/3.8.13/x64/bin/python",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "ELF",
            "machine": "x86_64",
            "processor": "x86_64",
            "node": "fv-az173-750",
            "version": "#37~20.04.1-Ubuntu SMP Mon Jun 13 22:51:01 UTC 2022",
            "system": "Linux",
            "release": "5.13.0-1031-azure",
            "libcver": [


2022-07-04 14:49:25,384 DEBUG [__main__.mytool] (application.exit): Exiting application: mytool


see what happens when a value is set that is not of the correct type:

In [16]:
try:
    tool.iterations = "badval"
except TraitError as E:
    print("bad value:",E)
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

bad value: The 'iterations' trait of a MyTool instance expected an int, not the str 'badval'.


Example of what happens when you change a parameter that is being "observed" in a class. It's handler is called:

In [17]:
tool.advanced.outfile = "Another.txt"

2022-07-04 14:49:25,403 WARNING [__main__.mytool.AdvancedComponent] (539495868.on_outfile_changed): Outfile was changed to '{'name': 'outfile', 'old': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/out.csv'), 'new': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/Another.txt'), 'owner': <__main__.AdvancedComponent object at 0x7f5d968609a0>, 'type': 'change'}'


we see that the handler for `outfile` was called, and it receive a change dict that shows the old and new values.

create a tool using a config file:

In [18]:
tool2 = MyTool()

In [19]:
try:
    tool2.run(argv=['--config', 'Tools.json'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2022-07-04 14:49:25,425 INFO [__main__.mytool] (tool.initialize): ctapipe version 0.15.1.dev142+ge09acbd4


2022-07-04 14:49:25,426 INFO [__main__.mytool] (tool.run): Starting: mytool


2022-07-04 14:49:25,439 WARNING [__main__.mytool.AdvancedComponent] (539495868.on_outfile_changed): Outfile was changed to '{'name': 'outfile', 'old': traitlets.Undefined, 'new': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/foo.txt'), 'owner': <__main__.AdvancedComponent object at 0x7f5d9453fc70>, 'type': 'change'}'


2022-07-04 14:49:25,442 DEBUG [__main__.mytool] (tool.run): CONFIG: {'MyTool': {'config_files': [PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/Tools.json')], 'iterations': 5, 'log_config': {}, 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_file': None, 'log_file_level': 'INFO', 'log_format': '[%(name)s]%(highlevel)s %(message)s', 'log_level': 10, 'logging_config': {}, 'provenance_log': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/mytool.provenance.log'), 'quiet': False, 'show_config': False, 'show_config_json': False, 'MyComponent': {'value': -1}, 'SecondaryMyComponent': {'value': -1}, 'TelescopeWiseComponent': {'param': [('type', '*', 5.0)]}, 'AdvancedComponent': {'infile': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/something.txt'), 'outfile': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/foo.txt'), 'value1': -1, 'MyComponent': {'value': -1}}}}


2022-07-04 14:49:25,444 INFO [__main__.mytool] (1473845174.start): Performing 5 iterations...


...

2022-07-04 14:49:25,969 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:25,969 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:25,970 INFO [__main__.mytool] (tool.write_provenance): Output: 


2022-07-04 14:49:25,974 DEBUG [__main__.mytool] (tool.write_provenance): PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "d909fb92-6c63-4afd-a6af-21b59852e3ed",
      "start": {
         "time_utc": "2022-07-04T14:49:23.726"
      },
      "stop": {
         "time_utc": "2022-07-04T14:49:24.250"
      },
      "system": {
         "ctapipe_version": "0.15.1.dev142+ge09acbd4",
         "ctapipe_resources_version": "not installed",
         "eventio_version": "1.9.1",
         "ctapipe_svc_path": null,
         "executable": "/opt/hostedtoolcache/Python/3.8.13/x64/bin/python",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "ELF",
            "machine": "x86_64",
            "processor": "x86_64",
            "node": "fv-az173-750",
            "version": "#37~20.04.1-Ubuntu SMP Mon Jun 13 22:51:01 UTC 2022",
            "system": "Linux",
            "release": "5.13.0-1031-azure",
            "libcver": [


2022-07-04 14:49:25,986 DEBUG [__main__.mytool] (application.exit): Exiting application: mytool


In [20]:
print(tool2.advanced.infile)

/home/runner/work/ctapipe/ctapipe/docs/examples/something.txt


In [21]:
print(tool2.config)

{'MyTool': {'config_files': ['Tools.json'], 'log_level': 'DEBUG'}, 'AdvancedComponent': {'infile': 'something.txt', 'outfile': 'foo.txt'}, 'attach_subarray': <LazyConfigValue {}>}


In [22]:
tool2.is_setup

True

In [23]:
tool3 = MyTool()

In [24]:
tool3.is_setup

False

In [25]:
tool3.initialize(argv=[])

In [26]:
tool3.is_setup

False

In [27]:
tool3

config_files,[],(default: traitlets.Undefined)
iterations,5,Number of times to run (default: 5)
log_config,{},(default: traitlets.Undefined)
log_datefmt,%Y-%m-%d %H:%M:%S,The date format used by logging formatters for %(asctime)s (default: %Y-%m-%d %H:%M:%S)
log_file,None,Filename for the log (default: None)
log_file_level,INFO,Logging Level for File Logging (default: INFO)
log_format,[%(name)s]%(highlevel)s %(message)s,The Logging format template (default: [%(name)s]%(highlevel)s %(message)s)
log_level,30,Set the log level by value or name. (default: 30)
logging_config,{},"Configure additional log handlers. The default stderr logs handler is configured by the log_level, log_datefmt and log_format settings. This configuration can be used to configure additional handlers (e.g. to output the log to a file) or for finer control over the default handlers. If provided this should be a logging configuration dictionary, for more information see: https://docs.python.org/3/library/logging.config.html#logging-config-dictschema This dictionary is merged with the base logging configuration which defines the following: * A logging formatter intended for interactive use called ``console``. * A logging handler that writes to stderr called ``console`` which uses the formatter ``console``. * A logger with the name of this application set to ``DEBUG`` level. This example adds a new handler that writes to a file: .. code-block:: python c.Application.logging_config = { 'handlers': { 'file': { 'class': 'logging.FileHandler', 'level': 'DEBUG', 'filename': '', } }, 'loggers': { '': { 'level': 'DEBUG', # NOTE: if you don't list the default ""console"" # handler here then it will be disabled 'handlers': ['console', 'file'], }, } } (default: traitlets.Undefined)"
provenance_log,/home/runner/work/ctapipe/ctapipe/docs/examples/mytool.provenance.log,(default: traitlets.Undefined)
quiet,False,(default: False)


In [28]:
tool.setup()
tool

2022-07-04 14:49:26,064 WARNING [__main__.mytool.AdvancedComponent] (539495868.on_outfile_changed): Outfile was changed to '{'name': 'outfile', 'old': traitlets.Undefined, 'new': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/out.csv'), 'owner': <__main__.AdvancedComponent object at 0x7f5d96860fa0>, 'type': 'change'}'


config_files,[],(default: traitlets.Undefined)
iterations,3,Number of times to run (default: 5)
log_config,{},(default: traitlets.Undefined)
log_datefmt,%Y-%m-%d %H:%M:%S,The date format used by logging formatters for %(asctime)s (default: %Y-%m-%d %H:%M:%S)
log_file,None,Filename for the log (default: None)
log_file_level,INFO,Logging Level for File Logging (default: INFO)
log_format,%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s,The Logging format template (default: [%(name)s]%(highlevel)s %(message)s)
log_level,10,Set the log level by value or name. (default: 30)
logging_config,{},"Configure additional log handlers. The default stderr logs handler is configured by the log_level, log_datefmt and log_format settings. This configuration can be used to configure additional handlers (e.g. to output the log to a file) or for finer control over the default handlers. If provided this should be a logging configuration dictionary, for more information see: https://docs.python.org/3/library/logging.config.html#logging-config-dictschema This dictionary is merged with the base logging configuration which defines the following: * A logging formatter intended for interactive use called ``console``. * A logging handler that writes to stderr called ``console`` which uses the formatter ``console``. * A logger with the name of this application set to ``DEBUG`` level. This example adds a new handler that writes to a file: .. code-block:: python c.Application.logging_config = { 'handlers': { 'file': { 'class': 'logging.FileHandler', 'level': 'DEBUG', 'filename': '', } }, 'loggers': { '': { 'level': 'DEBUG', # NOTE: if you don't list the default ""console"" # handler here then it will be disabled 'handlers': ['console', 'file'], }, } } (default: traitlets.Undefined)"
provenance_log,/home/runner/work/ctapipe/ctapipe/docs/examples/mytool.provenance.log,(default: traitlets.Undefined)
quiet,False,(default: False)


In [29]:
tool.comp2

AttributeError: 'NoneType' object has no attribute 'expandtabs'

## Getting the configuration of an instance

In [30]:
tool.get_current_config()

{'MyTool': {'config_files': [],
  'iterations': 3,
  'log_config': {},
  'log_datefmt': '%Y-%m-%d %H:%M:%S',
  'log_file': None,
  'log_file_level': 'INFO',
  'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s',
  'log_level': 10,
  'logging_config': {},
  'provenance_log': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/mytool.provenance.log'),
  'quiet': False,
  'show_config': False,
  'show_config_json': False,
  'MyComponent': {'value': -1},
  'SecondaryMyComponent': {'value': -1},
  'TelescopeWiseComponent': {'param': TelescopePatternList([('type',
                          '*',
                          5.0)])},
  'AdvancedComponent': {'infile': PosixPath('/home/runner/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4/gamma_test.simtel.gz'),
   'outfile': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/out.csv'),
   'value1': -1,
   'MyComponent': {'value': -1}}}}

In [31]:
tool.iterations = 12
tool.get_current_config()

{'MyTool': {'config_files': [],
  'iterations': 12,
  'log_config': {},
  'log_datefmt': '%Y-%m-%d %H:%M:%S',
  'log_file': None,
  'log_file_level': 'INFO',
  'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s',
  'log_level': 10,
  'logging_config': {},
  'provenance_log': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/mytool.provenance.log'),
  'quiet': False,
  'show_config': False,
  'show_config_json': False,
  'MyComponent': {'value': -1},
  'SecondaryMyComponent': {'value': -1},
  'TelescopeWiseComponent': {'param': TelescopePatternList([('type',
                          '*',
                          5.0)])},
  'AdvancedComponent': {'infile': PosixPath('/home/runner/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4/gamma_test.simtel.gz'),
   'outfile': PosixPath('/home/runner/work/ctapipe/ctapipe/docs/examples/out.csv'),
   'value1': -1,
   'MyComponent': {'value': -1}}}}

## Writing a Sample Config File

In [32]:
print(tool.generate_config_file())

# Configuration file for mytool.

#------------------------------------------------------------------------------
# Application(SingletonConfigurable) configuration
#------------------------------------------------------------------------------
## This is an application.

## The date format used by logging formatters for %(asctime)s
#  Default: '%Y-%m-%d %H:%M:%S'
# c.Application.log_datefmt = '%Y-%m-%d %H:%M:%S'

## The Logging format template
#  Default: '[%(name)s]%(highlevel)s %(message)s'
# c.Application.log_format = '[%(name)s]%(highlevel)s %(message)s'

## Set the log level by value or name.
#  Choices: any of [0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL']
#  Default: 30
# c.Application.log_level = 30

## Configure additional log handlers.
#  
#  The default stderr logs handler is configured by the log_level, log_datefmt
#  and log_format settings.
#  
#  This configuration can be used to configure additional handlers (e.g. to
#  output the log to a file) 